# Sequence-to-Sequence Model (Seq2Seq) para Traducción de Idiomas

En esta notebook, exploraremos la implementación de un modelo Sequence-to-Sequence (Seq2Seq) utilizando PyTorch. Este tipo de modelos es ampliamente utilizado en tareas de Procesamiento de Lenguaje Natural (NLP) que requieren el manejo de secuencias de texto de longitud variable, como la traducción automática, el resumen de textos y la generación de lenguaje natural.

## Introducción

### Objetivos

1. **Entender el funcionamiento de un modelo Seq2Seq** y cómo se aplica en la traducción automática de idiomas.
2. **Implementar el pipeline completo de un modelo Seq2Seq en PyTorch**, desde la preparación de los datos hasta el entrenamiento y evaluación del modelo.
3. **Explorar el uso del `teacher forcing`** para mejorar la eficiencia en el entrenamiento de redes neuronales recurrentes.

### Contenido

1. Introducción a los modelos Sequence-to-Sequence (Seq2Seq) y su relevancia en tareas de traducción automática.
2. Preparación de datos textuales para alimentar a un modelo Seq2Seq.
3. Implementación de un **encoder-decoder** utilizando LSTM para la traducción de secuencias.
4. Aplicación de **teacher forcing** durante el entrenamiento para mejorar el rendimiento.
5. Evaluación del modelo y generación de traducciones en el conjunto de prueba.

### Concepto de Seq2Seq

Un modelo Seq2Seq consiste en dos partes principales: un **encoder** y un **decoder**. El encoder procesa la secuencia de entrada y la comprime en un vector de estado oculto, que luego es utilizado por el decoder para generar la secuencia de salida.

- **Encoder**: Procesa la secuencia de entrada palabra por palabra y genera un conjunto de estados ocultos que resumen el contenido de la secuencia.
- **Decoder**: Utiliza el estado final del encoder para generar la secuencia de salida, también palabra por palabra.

Este enfoque es particularmente útil en tareas de traducción automática, donde queremos convertir una oración en un idioma a una oración equivalente en otro idioma.

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-24_at_7.47.32_PM.png"/>

### Dataset de Traducción

Para esta notebook, utilizaremos un dataset de traducción inglés-español que contiene pares de frases. El objetivo es traducir frases de inglés a español, simulando un escenario de traducción automática. El dataset se compone de pares de frases cortas en ambos idiomas y servirá como base para entrenar el modelo Seq2Seq.

El mismo se encuentra disponible en el siguiente enlace: [Link al dataset](https://tatoeba.org/en/downloads)

### Referencias

- [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) - Sutskever et al. (2014)
- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078) - Cho et al. (2014)

In [1]:
!pip install torchinfo

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

from torchinfo import summary

import numpy as np
import os
import re
from pathlib import Path
from collections import Counter

In [3]:
# Fijamos la semilla para que los resultados sean reproducibles
SEED = 23

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
import sys

# definimos el dispositivo que vamos a usar
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS
elif torch.xpu.is_available():
    DEVICE = "xpu"  # si no hay GPU, pero hay XPU, usamos XPU

print(f"Usando {DEVICE}")

NUM_WORKERS = 0  # Win y MacOS pueden tener problemas con múltiples workers
if sys.platform == "linux":
    NUM_WORKERS = 4  # numero de workers para cargar los datos (depende de cada caso)

print(f"Usando {NUM_WORKERS}")

Usando cpu
Usando 4


## Cargar los Datos & Preprocesamiento

Vamos a leer el dataset de traducción y realizar un preprocesamiento básico para limpiar y normalizar los textos antes de alimentarlos al modelo Seq2Seq. Por ejemplo, vamos a convertir los textos a minúsculas, eliminar algunos caracteres especiales y filtrar las oraciones más largas.

In [5]:
def clean_text(text):
    # Convertimos a minúsculas
    text = text.lower()

    # Insertamos espacios alrededor de los símbolos de puntuación que queremos conservar
    text = re.sub(r"([¿?¡!])", r" \1 ", text)

    # Eliminamos todo lo que no sea letras, números, o los símbolos que queremos conservar
    text = re.sub(r"[^a-zA-Z0-9áéíóúüñ¿?¡!]+", " ", text)

    # Remover espacios extras
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [6]:
DATA_PATH = str(Path("data") / "English-Spanish.tsv")

MAX_SENTENCE_LENGTH = 8  # Máxima longitud de las frases que vamos a considerar


def load_data(source_file, max_words=5):
    with open(source_file, "r") as f:
        lines = f.readlines()

    # Separamos las frases en dos listas
    input_texts = []
    target_texts = []

    for line in lines:
        elements = line.split("\t")

        input_text = elements[1]
        target_text = elements[3]

        input_text_clean = clean_text(input_text)
        target_text_clean = clean_text(target_text)

        # Filtramos frases de hasta max_words palabras
        if (
            len(input_text_clean.split()) <= max_words
            and len(target_text_clean.split()) <= max_words
        ):
            input_texts.append(input_text_clean)
            target_texts.append(target_text_clean)

    return input_texts, target_texts


src_texts, trg_texts = load_data(DATA_PATH, MAX_SENTENCE_LENGTH)

In [7]:
print(f"Number of samples: {len(src_texts)}")

Number of samples: 193620


In [8]:
random_idx = np.random.randint(0, len(src_texts), 10)
for idx in random_idx:
    print(f"Input: {src_texts[idx]}")
    print(f"Target: {trg_texts[idx]}\n")

Input: our guests are in a hurry
Target: nuestros huéspedes tienen prisa

Input: did he do such things ?
Target: ¿ seguro que hizo esas cosas ?

Input: this room is large enough
Target: este cuarto es lo suficientemente grande

Input: is tatoeba a pamphlet or what ?
Target: ¿ tatoeba es un panfleto o qué ?

Input: i would like batteries for this device
Target: quisiera pilas para este aparato

Input: i want to show you something great
Target: quiero mostrarte algo grandioso

Input: the bad weather prevented me from going fishing
Target: el mal clima me impidió ir a pescar

Input: i ve acknowledged my mistake
Target: he reconocido mi error

Input: you screamed
Target: gritaron

Input: tom said mary did that on purpose
Target: tom dijo que mary lo hizo a propósito



## Construcción de los Vocabularios

Es importante construir un vocabulario para cada idioma en el dataset, ya que cada vocabulario tiene que ser capaz de mapear palabras a índices enteros y viceversa.

> Nota: para reducir el tiempo de entrenamiento, vamos a limitar el tamaño del vocabulario a las palabras más comunes en cada idioma, con el argumento `FREQ_THRESHOLD` controlamos la cantidad de palabras que se incluirán en el vocabulario.

Tenemos además que agregar token especiales:

- `SOS` (Start of Sentence): Indica el inicio de una oración.
- `EOS` (End of Sentence): Indica el final de una oración.
- `UNK` (Unknown): Indica una palabra desconocida que no está en el vocabulario.
- `PAD` (Padding): Se utiliza para rellenar secuencias a la misma longitud.

In [9]:
PAD_TOKEN = "<PAD>"
SOS_TOKEN = "<SOS>"
EOS_TOKEN = "<EOS>"
UNK_TOKEN = "<UNK>"
FREQ_THRESHOLD = 3  # Frecuencia mínima para considerar una palabra en el vocabulario

class Vocab:
    def __init__(self):
        # mapea palabras a índices
        self.word2index = {}
        # mapea índices a palabras
        self.index2word = {}
        # autonumeración de índices
        self.index = 0

        # Tokens especiales
        self.add_special_tokens()

    def add_special_tokens(self):
        self.add_word(PAD_TOKEN)
        self.add_word(SOS_TOKEN)
        self.add_word(EOS_TOKEN)
        self.add_word(UNK_TOKEN)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.index
            self.index2word[self.index] = word
            self.index += 1

    def build_vocab(self, sentences, min_freq=1):
        word_counter = Counter()
        for sentence in sentences:
            for word in sentence.split():
                word_counter[word] += 1

        # Filtrar palabras que no alcanzan la frecuencia mínima
        words = [word for word, count in word_counter.items() if count >= min_freq]

        # Agregar palabras filtradas al vocabulario
        for word in words:
            self.add_word(word)

    def __len__(self):
        return len(self.word2index)

    def __getitem__(self, key):
        if isinstance(key, int):
            return self.index2word.get(key, UNK_TOKEN)
        if isinstance(key, str):
            return self.word2index.get(key, self.word2index[UNK_TOKEN])


# Construimos los vocabularios
SRC_VOCAB = Vocab()
TRG_VOCAB = Vocab()

SRC_VOCAB.build_vocab(src_texts, min_freq=FREQ_THRESHOLD)
TRG_VOCAB.build_vocab(trg_texts, min_freq=FREQ_THRESHOLD)

SRC_VOCAB_SIZE = len(SRC_VOCAB)
TRG_VOCAB_SIZE = len(TRG_VOCAB)

print(f"English vocab size: {SRC_VOCAB_SIZE}")
print(f"Spanish vocab size: {TRG_VOCAB_SIZE}")

English vocab size: 9832
Spanish vocab size: 15208


Algunos ejemplos de uso de los vocabularios:

- `SRC_VOCAB['hello']`: Devuelve el índice de la palabra "hello" en el vocabulario de origen.
- `TGT_VOCAB['hola']`: Devuelve el índice de la palabra "hola" en el vocabulario de destino.
- `TRG_VOCAB['palabra_no_existente']`: Devuelve el índice de la palabra desconocida (`<UNK>`) en el vocabulario de destino.
- `TRG_VOCAB[10]`: Devuelve la palabra en el índice 10 del vocabulario de destino.
- `TRG_VOCAB[3]`: Devuelve la palabra en el índice 3 del vocabulario de destino.
- `SRC_VOCAB[PAD_TOKEN]`: Devuelve el índice del token de padding en el vocabulario de origen.


In [17]:
print(SRC_VOCAB["hello"])
print(TRG_VOCAB["hola"])
print(TRG_VOCAB["palabra_no_existente"])
print(TRG_VOCAB[15])
print(SRC_VOCAB[14])
print(TRG_VOCAB[PAD_TOKEN])

550
695
3
tiene
is
0


In [16]:
# Función para codificar una frase (strings -> índices)
def encode_sentence(sentence, vocab):
    return [vocab[word] for word in sentence.split()]

print(encode_sentence("hello world", SRC_VOCAB))
print(encode_sentence("hola mundo extraterrestre", TRG_VOCAB))

[550, 51]
[695, 53, 3]


In [21]:
# Función para decodificar una secuencia de índices (indices -> strings)
def decode_sentence(indices, vocab):
    if isinstance(indices, torch.Tensor): # en caso que nos pasen un tensor
        indices = indices.tolist()
    return " ".join([vocab[idx] for idx in indices if idx != vocab[PAD_TOKEN] and idx != vocab[EOS_TOKEN] and idx != vocab[SOS_TOKEN] and idx != vocab[UNK_TOKEN]])

print(
    decode_sentence([TRG_VOCAB[SOS_TOKEN],10, 11, 12, TRG_VOCAB[PAD_TOKEN], TRG_VOCAB[PAD_TOKEN], TRG_VOCAB[EOS_TOKEN]], TRG_VOCAB)
)

print(
    decode_sentence(torch.randint(0, len(TRG_VOCAB), (3,)), TRG_VOCAB)
)

irme a dormir
desapareció fiable haumea


## Dataset de Traducción

Trabajaremos con un pequeño dataset de traducción **inglés-español**, compuesto por pares de frases simples. El objetivo es que el modelo aprenda a traducir una frase en inglés a su equivalente en español.

#### Ejemplos de Pares:

- **Inglés**: "hello" → **Español**: "hola"
- **Inglés**: "how are you?" → **Español**: "¿cómo estás?"

### Preparación del Dataset

Cada frase será tokenizada y convertida a índices numéricos de sus respectivos vocabularios. En las secuencias objetivo, añadimos los tokens especiales `<SOS>` y `<EOS>` para marcar el inicio y el fin de cada traducción.

In [36]:
class TranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences):
        super(TranslationDataset, self).__init__()
        self.source_sentences = source_sentences
        self.target_sentences = target_sentences

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, idx):
        src = self.source_sentences[idx] # la frase con palabras src
        trg = self.target_sentences[idx] # la frase con palabras trg

        src = encode_sentence(src, SRC_VOCAB)
        trg = encode_sentence(trg, TRG_VOCAB)

        trg = [TRG_VOCAB[SOS_TOKEN]] + trg + [TRG_VOCAB[EOS_TOKEN]]

        src = torch.tensor(src, dtype=torch.long)
        trg = torch.tensor(trg, dtype=torch.long)

        return src, trg

# Crear el dataset y el dataloader
train_dataset = TranslationDataset(src_texts, trg_texts)
val_len = int(0.10 * len(train_dataset))
train_len = len(train_dataset) - val_len
train_dataset, val_dataset = random_split(train_dataset, [train_len, val_len])

In [37]:
# probamos algunos ejemplos
random_idx = np.random.randint(
    0, len(train_dataset), 5
)  # tomamos 5 ejemplos del dataset train

for idx in random_idx:
    x, y = train_dataset[idx]
    x_sentence = decode_sentence(x, SRC_VOCAB)  # int -> palabras
    y_sentence = decode_sentence(y, TRG_VOCAB)  # int -> palabras
    print(f"SRC:           {x_sentence}")
    print(f"SRC (encoded): {x}")
    print(f"TRG:           {y_sentence}")
    print(f"TRG (encoded): {y}\n")

SRC:           who invented the first automobile ?
SRC (encoded): tensor([ 293,  389,   17,  348, 4274,   49])
TRG:           ¿ quién inventó el primer automóvil ?
TRG (encoded): tensor([   1,   46,  356, 3587,  275, 3306, 5468,   50,    2])

SRC:           may his soul rest in peace
SRC (encoded): tensor([ 191,  599,  789,  949,   44, 2346])
TRG:           que en paz descanse
TRG (encoded): tensor([    1,     9,    24,  3269, 10702,     2])

SRC:           tom is insolent
SRC (encoded): tensor([ 485,   14, 8405])
TRG:           tom es insolente
TRG (encoded): tensor([    1,   618,    20, 12861,     2])

SRC:           dark matter is invisible
SRC (encoded): tensor([ 605, 2181,   14, 1457])
TRG:           la materia oscura es invisible
TRG (encoded): tensor([   1,   18, 4668, 4041,   20, 2041,    2])

SRC:           the roof of my house is red
SRC (encoded): tensor([  17, 2582,   91,   60, 1307,   14,  427])
TRG:           el techo de mi casa es rojo
TRG (encoded): tensor([   1,  275, 

Debido a que las RNNs requieren secuencias de longitud fija, vamos a rellenar las secuencias con el token especial `<PAD>` para que todas tengan la misma longitud. Para eso podemos auxiliarnos de la función [pad_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html) de PyTorch.

La función `collate_fn` es un argumento opcional que se pasa al DataLoader de PyTorch para personalizar el procesamiento de los datos. En este caso, se utiliza para rellenar y agrupar las secuencias de entrada y salida en lotes.

In [38]:
seq_1 = torch.tensor([1, 2, 3, 4])
seq_2 = torch.tensor([1, 2, 3])
seq_3 = torch.tensor([1, 2, 3, 4, 5, 6])
padded_seqs = pad_sequence([seq_1, seq_2, seq_3], batch_first=True, padding_value=0, padding_side='left')
print(padded_seqs)

tensor([[0, 0, 1, 2, 3, 4],
        [0, 0, 0, 1, 2, 3],
        [1, 2, 3, 4, 5, 6]])


In [46]:
def collate_fn(batch):
    sources, targets = zip(*batch)

    sources = pad_sequence(
        sources, batch_first=True, padding_value=SRC_VOCAB[PAD_TOKEN], padding_side='right'
    )

    targets = pad_sequence(
        targets, batch_first=True, padding_value=TRG_VOCAB[PAD_TOKEN], padding_side='right'
    )

    return sources, targets

In [47]:
BATCH_SIZE = 512

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
)
valid_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)

In [48]:
x, y = next(iter(train_loader))
print(f"Source batch shape: {x.shape}")
print(f"Target batch shape: {y.shape}")

Source batch shape: torch.Size([512, 8])
Target batch shape: torch.Size([512, 10])


## Modelo Seq2Seq

El modelo Seq2Seq que vamos a implementar consiste en un **encoder** y un **decoder** basados en capas LSTM. El encoder procesa la secuencia de entrada y genera un vector de contexto que es utilizado por el decoder para generar la secuencia de salida.

<img src="https://docs.chainer.org/en/v7.8.0/_images/seq2seq.png">

In [49]:
# Definicion de hiperparámetros del modelo y entrenamiento
EMBEDDING_DIM = 128
HIDDEN_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5

LR = 0.001
EPOCHS = 20

INITIAL_TEACHER_FORCING_RATIO = 1.0
TEACHER_FORCING_DECAY = 0.95
MIN_TEACHER_FORCING_RATIO = 0.3

### Encoder

El encoder procesa la secuencia de entrada palabra por palabra y genera una representación de la secuencia en forma de un vector de contexto.

- Primero, la secuencia de entrada es pasada a través de una capa de embedding para convertir las palabras en vectores densos.
- Luego, los embeddings de palabras son pasados a una capa LSTM que procesa la secuencia y genera una representación de la secuencia en forma de un vector de estado oculto.
- La salida del encoder es el estado oculto final y la celda oculta final de la capa LSTM.

In [51]:
class Encoder(nn.Module):
    def __init__(self, emb_dim, hidden_dim, n_layers, dropout):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(SRC_VOCAB_SIZE, emb_dim,SRC_VOCAB[PAD_TOKEN])
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, n_layers, batch_first = True, dropout=dropout)

    def forward(self, src):
        # src: [batch_size, seq_len]
        embed = self.dropout(self.embed(src))
        # embed: [batch_size, seq_len, emb_dim]
        _, (hidden, cell) = self.lstm(embed)
        # hidden: [num_layer, batch_size, hidden_dim]
        # cell: [num_layer, batch_size, hidden_dim]
        return hidden, cell

summary(
    Encoder(EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT),
    input_size=(BATCH_SIZE, MAX_SENTENCE_LENGTH),
    dtypes=[torch.long],
)

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [2, 512, 512]             --
├─Embedding: 1-1                         [512, 8, 128]             1,258,496
├─Dropout: 1-2                           [512, 8, 128]             --
├─LSTM: 1-3                              [512, 8, 512]             3,416,064
Total params: 4,674,560
Trainable params: 4,674,560
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 14.64
Input size (MB): 0.03
Forward/backward pass size (MB): 20.97
Params size (MB): 18.70
Estimated Total Size (MB): 39.70

### Decoder

El decoder genera la secuencia de salida palabra por palabra utilizando el vector de contexto generado por el encoder. A diferencia del encoder, el decoder es entrenado para predecir la siguiente palabra en la secuencia de salida.

- Se recibe el input (token de entrada) y el estado oculto del encoder (hidden, cell).
- El token de entrada es pasado a través de una capa de embedding para convertirlo en un vector denso.
- Luego, el embedding de la palabra y el estado oculto anterior son usado para predecir la siguiente palabra en la secuencia de salida.
- Se retorna la predicción y el nuevo estado oculto.

In [52]:
class Decoder(nn.Module):
    def __init__(self, emb_dim, hidden_dim, n_layers, dropout):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(TRG_VOCAB_SIZE, emb_dim,TRG_VOCAB[PAD_TOKEN])
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, n_layers, batch_first = True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, TRG_VOCAB_SIZE)

    def forward(self, input_token, hidden, cell):
        # input_token: [batch_size] -> un solo token por cada oración en el batch
        # hidden: [n_layers, batch_size, hidden_dim]
        # cell: [n_layers, batch_size, hidden_dim]
        input_token = torch.unsqueeze(input_token, 1)
        # input_token: [batch_size, 1]
        embed = self.dropout(self.embed(input_token))
        # embed: [batch_size, 1, emb_dim]
        output, (hidden, cell) = self.lstm(embed, (hidden, cell))
        # output: [batch_size, 1, hidden_dim]
        output = torch.squeeze(output, 1)
        # output: [batch_size, hidden_dim]
        output = self.fc(output)
        # prediction: [batch_size, trg_vocab_size]
        return output, hidden, cell


summary(
    Decoder(EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT),
    input_size=[
        (BATCH_SIZE,),
        (N_LAYERS, BATCH_SIZE, HIDDEN_DIM),
        (N_LAYERS, BATCH_SIZE, HIDDEN_DIM),
    ],
    dtypes=[torch.long, torch.float, torch.float],
)

Layer (type:depth-idx)                   Output Shape              Param #
Decoder                                  [512, 15208]              --
├─Embedding: 1-1                         [512, 1, 128]             1,946,624
├─Dropout: 1-2                           [512, 1, 128]             --
├─LSTM: 1-3                              [512, 1, 512]             3,416,064
├─Linear: 1-4                            [512, 15208]              7,801,704
Total params: 13,164,392
Trainable params: 13,164,392
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 6.74
Input size (MB): 4.20
Forward/backward pass size (MB): 64.91
Params size (MB): 52.66
Estimated Total Size (MB): 121.77

## Entrenamiento del Modelo

Para entrenar el modelo, vamos a definir una función de pérdida y un optimizador. La función de pérdida será la entropía cruzada categórica, ya que estamos tratando con un problema de clasificación de múltiples clases. Además utilizaremos el parametro `ignore_index` del criterio de pérdida para no calcular la pérdida en los tokens de padding.

El optimizador necesita los parámetros del Encoder y del Decoder, así como la tasa de aprendizaje.


In [53]:
encoder = Encoder(EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(DEVICE)
decoder = Decoder(EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(DEVICE)

criterion = nn.CrossEntropyLoss(
    ignore_index=TRG_VOCAB[PAD_TOKEN], label_smoothing=0.05
)  # ignoramos el padding en el cálculo de la loss
optimizer = optim.Adam(
    list(encoder.parameters()) + list(decoder.parameters()), lr=LR
)  # tomamos los parámetros de ambos modelos

### Entrenamiento de una Época en Seq2Seq con *Teacher Forcing*

**Cálculo de la pérdida para cada token**

A diferencia de entrenamientos comunes donde se calcula la pérdida para toda la secuencia de una vez, aquí la pérdida se calcula **token por token** dentro de un bucle. Esto permite que el decoder procese un token a la vez, prediciendo el siguiente en función del estado oculto anterior.

**Uso de *Teacher Forcing***

Se introduce un parámetro **`teacher_forcing_ratio`**, que controla la probabilidad de usar el token real de la secuencia objetivo como siguiente entrada. Esto difiere de los enfoques comunes donde siempre se usa la predicción del modelo para generar la siguiente entrada.

**Actualización de gradientes**
La **retropropagación** ocurre después de que se procesan todos los tokens en la secuencia, acumulando las pérdidas de cada token antes de hacer `backward()` y actualizar los parámetros con `optimizer.step()`.

**Proceso resumido**

1. Inicialización de `encoder` y `decoder` en modo entrenamiento.
2. Para cada batch, se:
   - Calcula la pérdida token por token.
   - Decide si usar el token objetivo real o la predicción anterior.
3. Finalmente, se acumula la pérdida total por secuencia.


In [55]:
def process_batch(src, trg, encoder, decoder, criterion, teacher_forcing_ratio):
    # src: [batch_size, src_len]
    # trg: [batch_size, trg_len]
    trg_input = trg[:, :-1]
    trg_output = trg[:, 1:]
    # trg_input: [batch_size, trg_len -1]
    trg_len = trg_input.shape[1]
    batch_size = trg_input.shape[0]

    token_input = trg_input[:, 0]
    # token_input: [batch_size]

    hidden, cell = encoder(src)

    batch_loss = 0
    for i in range(trg_len):
        output, hidden, cell = decoder(
            token_input, hidden, cell
        )

        batch_loss += criterion(output, trg_output[:, i])

        if torch.rand(1).item() < teacher_forcing_ratio:
            token_input = trg_input[:, i]
        else:
            token_input = torch.argmax(output, dim=1).detach()

    return batch_loss / trg_len
        # decoder_output: [batch_size, trg_vocab_size]

In [56]:
def train_epoch(
    encoder, decoder, dataloader, optimizer, criterion, teacher_forcing_ratio
):
    encoder.train()
    decoder.train()
    epoch_loss = 0

    for src, trg in dataloader:
        src, trg = src.to(DEVICE), trg.to(DEVICE)
        optimizer.zero_grad()
        loss = process_batch(src, trg, encoder, decoder, criterion, teacher_forcing_ratio)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [57]:
def evaluate(encoder, decoder, valid_loader, criterion):
    encoder.eval()
    decoder.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, trg in valid_loader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)
            loss = process_batch(src, trg, encoder, decoder, criterion, 0)

            epoch_loss += loss.item()

    return epoch_loss / len(valid_loader)

In [58]:
def train(
    encoder,
    decoder,
    train_loader,
    valid_loader,
    optimizer,
    criterion,
    n_epochs,
    initial_teacher_forcing_ratio=INITIAL_TEACHER_FORCING_RATIO,
    teacher_forcing_decay=TEACHER_FORCING_DECAY,
    min_teacher_forcing_ratio=MIN_TEACHER_FORCING_RATIO,
):
    current_teacher_forcing_ratio = initial_teacher_forcing_ratio
    for epoch in range(n_epochs):
        train_loss = train_epoch(
            encoder,
            decoder,
            train_loader,
            optimizer,
            criterion,
            current_teacher_forcing_ratio,
        )
        val_loss = evaluate(encoder, decoder, valid_loader, criterion)
        print(
            f"Epoch {epoch + 1} FT: {current_teacher_forcing_ratio:.2f} Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f}"
        )
        current_teacher_forcing_ratio = max(
            min_teacher_forcing_ratio,
            current_teacher_forcing_ratio * teacher_forcing_decay,
        )

In [59]:
train(
    encoder, decoder, train_loader, valid_loader, optimizer, criterion, n_epochs=EPOCHS
)

KeyboardInterrupt: 

## Evaluación del Modelo

Vamos a traducir algunas frases de prueba utilizando el modelo entrenado y evaluar la calidad de las traducciones generadas. Para esto, vamos a implementar una función `translate_sentence` que toma una oración en inglés y la traduce al español.

Esta vez es el modelo quien decide cuando parar con en el token `<EOS>`, aunque también se puede limitar la longitud máxima de la traducción.

In [ ]:
def generate_sequence(src, encoder, decoder, max_len):
    encoder.eval()
    decoder.eval()
    generated_tokens = []
    with torch.no_grad():
        pass

    return generated_tokens

In [ ]:
def translate_sentence(encoder, decoder, sentence, max_len):
    sentence = clean_text(sentence)
    sentence_indexes = encode_sentence(sentence, SRC_VOCAB)
    sentence_tensor = torch.tensor(sentence_indexes).unsqueeze(0).to(DEVICE)
    generated_tokens = generate_sequence(sentence_tensor, encoder, decoder, max_len)
    predicted_sentence = decode_sentence(generated_tokens, TRG_VOCAB)
    return predicted_sentence

In [ ]:
sentences = [
    "I am hungry",
    "I am tired",
    "I am happy",
    "I'm sad",
    "I am angry",
    "every time I study, I get sleepy",
    "I am going to the gym",
    "I am going to the beach",
    "I am going to the supermarket",
    "I'm going to the movies",
    "I don't know what to do",
    "I love deep learning",
    "I can't open the door",
    "you can go if you want to",
    "i'm going to the party",
    "where does all this come from ?",
    "I can read your mind",
    "I can't believe it",
    "I can't believe you",
    "I can't believe this",
    "I didn't like it",
    "You can do it",
    "Do you speak Italian?",
    "Do you want to learn Spanish?",
    "I want to learn French",
    "Do you want to go to the movies?",
    "this is my favorite song",
    "I can't wait to see you",
    "see you later",
    "have a nice day",
    "we'll talk later",
    "let's grab a coffee sometime",
    "they're coming to the party",
    "she's my best friend",
    "he's a great guy",
    "My class is in 30 minutes.",
    "I have class tomorrow."
]

for sentence in sentences:
    print(f"Input: {sentence}")
    print(f"Translation: {translate_sentence(encoder, decoder, sentence, 12)}\n")

In [ ]:
translate_sentence(encoder, decoder, "we need attention", MAX_SENTENCE_LENGTH)

In [ ]:
translate_sentence(encoder, decoder, "let's go for it!", MAX_SENTENCE_LENGTH)